In [2]:
#from geojson import FeatureCollection
import geopandas


In [4]:
url_coral_reef = "https://opendata.arcgis.com/datasets/9229d814438349948c99b5e61a084418_7.geojson"

#my_feature = FeatureCollection(url_coral_reef)


In [8]:

gdf = geopandas.read_file(url_coral_reef)
gdf.explore("sourcethm", legend=True)